# Project: GW injection and Parameter estimation¶
Author: Tri Nguyen

In this tutorial, we will inject a gravitational-wave (GW) signal into LIGO data and perform parameter estimation on a redudced parameter space to extract the parameters of the signal. We will be utilizing the Bilby, which is a Python-based user-friendly Bayesian inference library for GW data analysis. For more information about Bilby, please visit: https://lscsoft.docs.ligo.org/bilby/ and https://arxiv.org/pdf/1811.02042.pdf. 


This notebook is based on the Bilby tutorial for GW injection and parameter estimation.

In [ ]:
# Install Bilby and some of its dependency
!pip install bilby
!pip install gwpy lalsuite

In [ ]:
from __future__ import division, print_function

import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

import bilby
from bilby.gw.source import lal_binary_black_hole
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters

%matplotlib inline

## Gravitational-wave injection
### Example

In the example, we will inject a binary black hole (BBH) into a Gaussian noise background.

In [ ]:
# Set random seed for reproducibility
np.random.seed(74656541)

In [ ]:
# We are going to inject a binary black hole waveform.  We first establish a
# dictionary of parameters that includes all of the different waveform
# parameters, including masses of the two black holes (mass_1, mass_2),
# spins of both black holes (a, tilt, phi), etc.
# make up some injection parameters and inject signal into data
injection_parameters = dict(
    mass_1=50, mass_2=50, a_1=0., a_2=0., tilt_1=0., tilt_2=0.,
    phi_12=0., phi_jl=0., luminosity_distance=500, theta_jn=0., psi=0.,
    phase=0.2, geocent_time=1243309096, ra=0., dec=0.)



# First, we define the duration and sampling frequency of the data segment
# that we will inject the GW signal into. For our BBH, a duration of 4 seconds 
# will be sufficient to capture the entire waveform.
duration = 4.
# We set the sampling frequency to 2048 Hz because the BBH waveform does not usually
# extend pass 1024 Hz 
sampling_frequency = 2048.

In [ ]:
# LIGO approximates GW waveforms by interpolating between GW templates in a template bank
# which is generated from physical simulations (e.g. numerical relativity, post-Newtonian). 
# These waveform families are called **waveform approximant**. 
# For more information on these waveform approximants, please visit: 

# https://www.lsc-group.phys.uwm.edu/ligovirgo/cbcnote/Waveforms/Overview

# In this example, we will use the waveform approximant `IMRPhenomPv2`. 
# This is a phenomenological model that approximates the GW waveforms 
# during the three stages of an inspiral (Inspiral, Merger, Ringdown). `IMRPhenomPv2` also uses 
# a single-spin frequency-dependent post-Newtonian rotation  to describe spin precession effects.

# We define a WaveformGenerator object to generate any BBH waveform 
# given the appropriate parameters
waveform_arguments = {
    'waveform_approximant': 'IMRPhenomPv2',
    'reference_frequency': 50.,  # most sensitive frequency
    'minimum_frequency': 20.
}
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration, sampling_frequency=sampling_frequency,
    parameter_conversion=convert_to_lal_binary_black_hole_parameters,
    frequency_domain_source_model=lal_binary_black_hole,
    waveform_arguments=waveform_arguments)

In [ ]:
# We'd like to see what kind of signal we will be generating
# We examine the signal in the time domain
polarizations_td = waveform_generator.time_domain_strain(injection_parameters)
plus_td = np.roll(polarizations_td['plus'], int(sampling_frequency * 2))
cross_td = np.roll(polarizations_td['cross'], int(sampling_frequency * 2))
time = np.linspace(0., duration, len(plus_td))

fig = plt.figure(figsize=(8, 5))

plt.plot(time, plus_td, label='Plus')
plt.plot(time, cross_td, label='Cross')
plt.xlabel('Time [sec]')
plt.ylabel('Strain')
plt.xlim(1.8, 2.02)
plt.legend()
plt.show()

# And their ASD
NFFT = int(4 * sampling_frequency)
freq, plus_psd = sig.welch(plus_td, fs=sampling_frequency, nperseg=NFFT)
plus_asd = np.sqrt(plus_psd)

freq, cross_psd = sig.welch(cross_td, fs=sampling_frequency, nperseg=NFFT)
cross_asd = np.sqrt(cross_psd)

fig = plt.figure(figsize=(8, 5))
plt.loglog(freq, plus_asd, label='Plus')
plt.loglog(freq, cross_asd, label='Cross')
plt.xlim(10, 1024)
plt.xlabel('Frequency [Hz]')
plt.ylabel('ASD')
plt.legend()
plt.show()

In [ ]:
# For this object, we will only use LIGO Hanford (H1) and LIGO Livingston (L1)
# Because H1 and L1 have different antenna patterns (which is based on their 
# orientation and location of Earth), each will see a different GW waveform. 
# We set up Bilby Interferometer object with the GPS time around the geocenter time
# of the GW signal. By default, each interferometer will have a Gaussian noise 
# background with the PSD being their design sensitivity.
ifos = bilby.gw.detector.InterferometerList(['H1', 'L1'])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency, duration=duration,
    start_time=injection_parameters['geocent_time'] - 3)

# Inject GW signal into H1 and L1
polarizations = ifos.inject_signal(waveform_generator=waveform_generator,
                                   parameters=injection_parameters)

In [ ]:
# We would like to visualize what the injected GW strain looks like
# First, we extract the H1 and L1 strain from the Inteferometer object
H1_strain = ifos[0].time_domain_strain
L1_strain = ifos[1].time_domain_strain

# And calculate their PSD and ASD
NFFT = int(sampling_frequency * 4)
freq, H1_strain_psd = sig.welch(H1_strain, fs=sampling_frequency, nperseg=NFFT)
H1_strain_asd = np.sqrt(H1_strain_psd)
freq, L1_strain_psd = sig.welch(L1_strain, fs=sampling_frequency, nperseg=NFFT)
L1_strain_asd = np.sqrt(L1_strain_psd)

# We also like to extract the background PSD and ASD
H1_bg_psd = ifos[0].power_spectral_density.psd_array
H1_bg_asd = np.sqrt(H1_bg_psd)
L1_bg_psd = ifos[1].power_spectral_density.psd_array
L1_bg_asd = np.sqrt(L1_bg_psd)
freq_bg = ifos[0].power_spectral_density.frequency_array

# We plot the strain ASD on top of the background ASD
# Because we inject a loud signal (H1 optimal SNR = 73.24, L1 optimal SNR = 79.75)
# we should be able to see the signal
fig = plt.figure(figsize=(12, 5))

# Plot H1 strain and background ASD
ax1 = plt.subplot(121)
ax1.loglog(freq, H1_strain_asd, label='Strain ASD')
ax1.loglog(freq_bg, H1_bg_asd, label='Background ASD')
ax1.set_title('H1')
ax1.set_xlim(20, 1024)
ax1.set_ylim(1e-26, None)
ax1.set_xlabel('Frequency')
ax1.set_ylabel('ASD')
ax1.legend()

# Plot L1 strain and background ASD
ax2 = plt.subplot(122)
ax2.loglog(freq, L1_strain_asd, label='Strain ASD')
ax2.loglog(freq_bg, L1_bg_asd, label='Background ASD')
ax2.set_title('L1')
ax2.set_xlim(20, 1024)
ax2.set_ylim(1e-26, None)
ax2.set_xlabel('Frequency')
ax2.set_ylabel('ASD')
ax2.legend()

plt.show()

### Student work

Now it is your time to shine. Inject a BBH with the following parameters into a Gaussian noise background:

- Masses $M_1$ and $M_2$: 36 $M_\odot$ and 29 $M_\odot$
- Dimensionless spin magnitudes $a_1$ and $a_2$: 0.4 and 0.3
- Difference between the azimuthal angles of the individual spin vector projections onto the orbital plane $\phi_{12}$: 97.4 $\deg$
- Difference between total and orbital angular momentum azimuthal angles $\phi_{jl}$: 17.2 $\deg$
- Tilt angle between their spins and the orbital angular momentum $\theta_1$ and $\theta_2$: 28.6 $\deg$ and 57.3 $\deg$
- Luminosity distance $D_L$: 2000 Mpc
- Inclination angle: $\theta_{jn}$: 22.9 $\deg$
- Polarisation angle $\psi$: 152.3 $\deg$
- Phase at coalescence $\phi_c$: 74.5 $\deg$
- Sky position RA and DEC: 78.8 $\deg$ and -69.4$\deg$
- Geocentric time: GPS 1126259642.413

You should use the same signal duration, sampling frequency, and waveform approximant as above. You do **not** have to reproduce the plots.

In [ ]:
%reset -f

In [ ]:
from __future__ import division, print_function

import numpy as np
import scipy.signal as sig
import matplotlib.pyplot as plt

import bilby
from bilby.gw.source import lal_binary_black_hole
from bilby.gw.conversion import convert_to_lal_binary_black_hole_parameters

%matplotlib inline

In [ ]:
# Set random seed for reproducibility
np.random.seed(74656541)

### -------------------START-CODE-------------------- ###

### -------------------END-CODE-------------------- ###

## Parameter estimation

We are now ready to extract the parameters of this GW signal from the Gaussian noise background. Our goal is to find the posterior distribution. However, because the posterior distribution is often intractable, LIGO uses posterior sampling algorithm (such as MCMC and Nested Sampling) to evaluate the posterior distribution. We will cover MCMC later in the course.


To begin sampling the posterior distribution, we need to define the prior distribution and likelihood distribution (Remember Bayes' rule?). We will first set up the prior distribution. Bilby uses a dictionary class structure to store the prior of each GW parameters. Each prior distribution can be defined with a Bilby class object. For more information about prior dictionary in Bilby, please visit: https://lscsoft.docs.ligo.org/bilby/prior.html.

Below is an example prior used to extract the parameters of the first BBH GW150914.

In [ ]:
example_priors = bilby.gw.prior.BBHPriorDict(filename='GW150914.prior')
example_priors

Now, you will create your own prior dictionary. By default, LIGO samples all 15 GW parameters (Can you name them all?). However, because this can take up to several days, we will instead work with a 2-dimensional space. Specifically, you will only sample the chirp mass $M_c$ and mass ratio $q$ of the BBH (see note). Feel free to play around with different type of priors.

You might find these Bilby classes useful:
- `bilby.prior.Uniform`
- `bilby.prior.Gaussian`
- `bilby.prior.DeltaFunction`
- ...

NOTE 1: We usually sample the chirp mass and mass ratio instead of the component masses $M_{1,2}$ because they tend to help the sampler converge faster and more easily. 

NOTE 2: You will have to define priors for the parameters that you are **not** sampling (e.g. $a_{1,2}$, $D_L$, etc.). If you don't, Bilby will use their default prior, and you will spend **a lot** of time sampling. Assume that we already know the values for these parameters, what kind of distribution should their priors be?

In [ ]:
priors = {}

### -------------------START-CODE-------------------- ###

### -------------------END-CODE-------------------- ###

priors

Now, we will set up a Gaussian noise likelihood function. Writing the likelihood can be a bit complicate, so we will use `bilby.gw.GravitationalWaveTransient` function from Bilby. However, it is important that you understand where this function , so we'll ask you to write its equation down.  

You can find your answer in this paper: https://arxiv.org/pdf/1809.02293.pdf (Hint: check the appendix)

#### Answer:
Remember to explain all your variables.
\begin{equation}
    \mathcal{L(d|\theta)} = 
\end{equation}
where $d$ is the GW strain and $\theta$ is a set of GW parameters.

In [ ]:
# Initialise the likelihood by passing in the interferometer data (ifos) and
# the waveform generator
likelihood = bilby.gw.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=waveform_generator)

Once we define the prior and likelihood distribution, we will use Bilby to sample the posterior distribution.

For this example, we will use the `dynesty` sampler, which Dynamic Nested Sampling package for estimating Bayesian posteriors and evidences. For more information about `dynesty`, please visit: https://dynesty.readthedocs.io/en/latest/.

In [ ]:
outdir = 'GW_PEInjection_result'
label = '2_parameters'
result = bilby.run_sampler(
    likelihood=likelihood, priors=priors, sampler='dynesty', npoints=1000, 
    resume=True, outdir=outdir, label=label)

Last but not least, let's show your sampling result by making a beautiful plot. The most common way to show a multi-dimensional posterior distribution is to use a corner plot. Plotting a corner plot manually can be tricky, so we strongly recommend you to use the `corner` package (https://corner.readthedocs.io/en/latest/). However, feel free to use any plotting tool you are accustomed to.

In this exercise, please plot the corner plot of the two component masses $M_1$ and $M_2$ of the BBH. Note that these are **not** the same parameters you have been sampling (which are chirp mass and mass ratio), so you will to convert your samples. You might find the chirp mass $M_c$ and mass ratio $q$ expressions below  useful:

\begin{equation}
M_c = \frac{(M_1 M_2)^{3/5}}{(M_1 + M_2)^{1/5}}\; \text{and} \; q=\frac{M_2}{M_1}
\end{equation}

What are the final values of the component masses $M_1$, $M_2$ of the BBH? Please include error bars and any neccessary statistics. How are these values compared to the true injection values?

In [ ]:
# First, we want to extract the posterior samples from bilby.result
samples = result.posterior[['chirp_mass', 'mass_ratio']].values

### -------------------START-CODE-------------------- ###

### -------------------END-CODE-------------------- ###

#### Answer:
The final values of the component masses $M_1$, $M_2$ of the BBH are:

## Conclusion

Congratulations! You have reached the end of the tutorial. Parameter estimation is a tough problem in gravitational-wave physics (and many other areas of physics) due to the intractable nature of the posterior distribution. Posterior sampling algorithms, such as MCMC and Nested Sampling, help solve this problem at a cost of long run-time. This makes it difficult to perform electromagnetic follow-up of time-sensitive GW sources like binary neutron stars. 

Deep learning algorithms are currently being developed to accelerate the posterior sampling process:

- https://arxiv.org/abs/2010.12931
- https://arxiv.org/abs/2002.07656
- https://arxiv.org/abs/2008.03312
- ...